# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

## Data Preparation (Loading CSV)

Load the `singapore_airline_reviews.csv` file into a pandas DataFrame `data`.

In [10]:
data = pd.read_csv("final_df.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11518 entries, 0 to 11517
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year                   11518 non-null  int64 
 1   month                  11518 non-null  int64 
 2   sentiment              11518 non-null  object
 3   processed_full_review  11518 non-null  object
dtypes: int64(2), object(2)
memory usage: 360.1+ KB


## Gated Recurrent Unit(GRU)

GRUs are primarily used neural networks in handling sequential data like text. They help the model to learn long-term dependencies in long texts, like remembering the context of earlier words in a sentence. This makes it especially effective in processing and predicting the sentiment of reviews. 

#### Common terms + definition:
Embeddings - vector of numbers that capture meaning and relationships between words. They also reduce high dimensionality of laguage, into something the computer can easily understand.
Embedding layer - Layer in deep learning models that learns these embeddings during training. 


In [ ]:
%pip install tensorflow
%pip install tensorflow-addons

In [12]:
%pip install scikeras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Imports specific to GRU
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1L2
from sklearn.metrics import classification_report



#### **Preparing data for GRU**

It is important to understand that GRUs are unable to understand words in the traditional sense; instead, the words need to be tokenized into numerical representations that the model can process(Here they are converted into integers). These tokenized words are typically mapped to a vocabulary where each unique word is assigned a specific integer value.

#### Tensorflow's Tokenizer or CountVectorization? :

Unlike CountVectorization, which uses a bag-of-words model(sparse matrix of word counts) where the order of words is ignored and not compatible with embedding layers, Tensorflow's Tokenizer is more suitable as it preserves the word order in sequences and is directly compatible with embedding layers.


In [17]:
# Extract features and labels
X = data['processed_full_review'].values  # The reviews (features)
y = data['sentiment'].values  # The sentiment labels

# Directly apply one-hot encoding
# Convert the categorical labels (e.g., "positive", "neutral", "negative") to one-hot encoded labels
y = pd.get_dummies(y).values  # Automatically converts to one-hot

# Tokenize and pad the text data
tokenizer = Tokenizer(num_words=5000)  # Limit to top 5000 words
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)

# Set a manageable padding length (e.g., 300 words)
max_sequence_len = 300
X_padded = pad_sequences(X_tokenized, maxlen=max_sequence_len)

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.3, random_state=42, stratify=np.argmax(y, axis=1))

# Compute class weights based on the training labels
y_train_labels = np.argmax(y_train, axis=1)  # Convert one-hot to single-label encoding for class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_labels), y=y_train_labels)
class_weights_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weights_dict)  # Check the weights calculated

Class Weights: {0: 1.5733801717408276, 1: 3.2973415132924337, 2: 0.4851657940663176}


#### **Building the GRU Model**
These integers are then fed into the GRU model. However, just converting words into integers is not enough to gain an understanding for their meaning. 

To give the model more context about the relationships between words, word embeddings (created by the Embedding layer in Keras) are used, which transform these integer tokens into dense vectors of real numbers that capture semantic meaning such as the similarity between words(Something the CountVectorizer cannot do).

#### Tensorflow's embedding or Word2Vec embedding? :
Word2Vec creates word embeddings __outside__ of the deep learning model (in this case we used Gensim), these embeddings are fixed and used directly in any downstream model. After training with Word2Vec, the embeddings are loaded into the model as __pre-trained embeddings__. This may potentially result in missing task-specific nuances(domain specific language).

The embedding layer in the tensorflow model is responsible for learning the word embeddings __during model training__. Hence the embeddings learned are __optimized for your dataset__ and the specific task at hand (such as sentiment classification). 


In [20]:
# Basic GRU model build
model = Sequential()

# Adding an Embedding layer to turn words into dense vectors
model.add(Embedding(input_dim=5000, output_dim=128))

# Add a GRU layer with 128 units(or neurons)
# The parameter units specifies the number of GRU neurons in this layer
# return_sequences = False tells the GRU to output only the final hidden state, more suitable for sentence classification
model.add(GRU(units=64, return_sequences=False,kernel_regularizer=L1L2(l1=0.001, l2=0.001))) #0.01 = 73.81%, loss: 0.8660

# Add a dropout layer to prevent overfitting
# By dropping neurons randomly while training, it ensures the model is not overly-reliant on a single neuron
model.add(Dropout(0.3))

# Output layer (for 3 classes: positive, neutral, negative)
model.add(Dense(3, activation='softmax'))

# Compile the model
# Categorical cross entropy is useful for multiclass problems (Calculates how far/close the predicted probability distribution is from the actual distribution of the target class)
# 'accuracy' is currently the key performance metric that is being tracked, perhaps can change to recall to make misclassifying negative more costly?

optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# Fit the model with specified number of epochs
history = model.fit(X_train, y_train, 
                    epochs=10,             # Number of epochs (you can adjust this number)
                    batch_size=32,          # Batch size
                    validation_split=0.2,   # 20% of the training data will be used for validation
                    class_weight=class_weights_dict)  # Optionally include class weights if the dataset is imbalanced


# Evaluate the optimized model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Test loss has no inherent meaning unless compared to other models, lower the better
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Make predictions on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

# Print classification report for detailed metrics per class
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"], digits=4))

Epoch 1/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.4547 - loss: 1.9972 - val_accuracy: 0.7514 - val_loss: 0.7551
Epoch 2/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.7814 - loss: 0.8051 - val_accuracy: 0.7254 - val_loss: 0.7129
Epoch 3/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.8379 - loss: 0.5853 - val_accuracy: 0.7588 - val_loss: 0.6560
Epoch 4/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.8740 - loss: 0.4528 - val_accuracy: 0.7750 - val_loss: 0.7103
Epoch 5/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.8819 - loss: 0.4270 - val_accuracy: 0.7316 - val_loss: 0.7760
Epoch 6/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.9186 - loss: 0.3188 - val_accuracy: 0.8010 - val_loss: 0.6772
Epoch 7/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.9384 - loss: 0.2567 - val_accuracy: 0.7929 - val_loss: 0.7487
Epoch 8/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9497 - loss: 0.2070 - val_accu

In [21]:
# GRU model with dynamic models
max_sequence_len = max([len(x) for x in X_tokenized])
def create_gru_model(input_dimensions=5000, output_dimensions=128, gru_units=128, dropout_rate=0.2, learning_rate=0.001):
    # Build the GRU model
    model = Sequential()   

    # Adding an Embedding layer to turn words into dense vectors
    # input_dim - Vocabulary size for the Embedding layer
    # output_dim - dimension of dense embedding vectors/number of numbers each word will be represented by 
    model.add(Embedding(input_dim=input_dimensions, output_dim=output_dimensions))

    # Add a GRU layer with 128 units(or neurons)
    # The parameter units specifies the number of GRU neurons in this layer
    # return_sequences = False tells the GRU to output only the final hidden state, more suitable for sentence classification
    model.add(GRU(units=gru_units, return_sequences=False))

    # Add a dropout layer to prevent overfitting
    # By dropping neurons randomly while training, it ensures the model is not overly-reliant on a single neuron
    model.add(Dropout(dropout_rate))

    # Output layer (for 3 classes: positive, neutral, negative)
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    # Categorical cross entropy is useful for multiclass problems(Calculates how far/close the predicted probability distribution is from the actual distribution of the target class)
    # 'accuracy' is currently the key performance metric that is being tracked, perhaps can change to recall to make misclassifying negative more costly?
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
 
    return model 


##### **Adam(Adaptive Moment Estimation):**
A popular optimization algorithm that adjusts the learning rate during training for each parameter. 

Key features:

1. Adaptive Learning Rates: Adam adjusts the learning rates for different parameters individually, allowing the model to converge faster and more effectively.

2. Momentum: Adam uses momentum to accelerate gradient descent, especially in the presence of noise or small gradients.

Adam is widely used because it combines the advantages of two other optimizers: AdaGrad(Algorithm that works well with spare gradients) and RMSProp(works well in non-stationary gradients).

#### GRU Cross Validation

In [22]:
# Parameter grid to test for cross validation 
param_grid = {
    'model__input_dimensions': [1000, 5000],
    'model__output_dimensions': [128],
    'model__gru_units': [128, 256],
    'model__dropout_rate': [0.1, 0.3],
    'model__learning_rate': [0.001, 0.01],
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

#### Finding the best parameters for the GRU Model

In [23]:
from sklearn.model_selection import GridSearchCV

# Wrap model function that was built above in a KerasClassifier
model = KerasClassifier(
    model=create_gru_model,
    input_dimensions=1000,  # default value
    output_dimensions=128,  # default value
    gru_units=128,         # default value
    dropout_rate=0.2,      # default value
    learning_rate=0.001,   # default value
    epochs=10,
    batch_size=32,
)

# Perform GridSearchCV using the Parameter Grid defined above 
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best results + use those results for training later 
print(f"Best Parameters: {grid_result.best_params_}")


ValueError: 
All the 192 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py", line 936, in _fit
  

Building the best model using gteh result of the best Parameters from GridSearchCV


In [ ]:
best_params = grid_result.best_params_
max_sequence_len = max([len(x) for x in X_tokenized])  # Find the longest review
# Unpack best parameters into create_gru_model function arguments
best_model = create_gru_model(
    input_dimensions=best_params['input_dimensions'],
    output_dimensions=best_params['output_dimensions'],
    max_sequence_len=max_sequence_len,  
    gru_units=best_params['gru_units'],
    dropout_rate=best_params['dropout_rate'],
    learning_rate=best_params['learning_rate'],
    num_classes = 3 )

optimizer = Adam(learning_rate=best_params['learning_rate'])
best_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Retrain model on the full training set
history = best_model.fit(
    X_train, y_train,
    epochs=best_params['epochs'],          # Best epoch count
    batch_size=best_params['batch_size'],   # Best batch size
    validation_data=(X_test, y_test)
)



#### Evaluating the model

In [ ]:
# Evaluate the optimized model
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

# Test loss has no inherent meaning unless compared to other models, lower the better
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


#### Classification Report

In [ ]:
from sklearn.metrics import classification_report

# Get the predicted classes for X_test
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Print the classification report
print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))